In [1]:
import pandas as pd
import re
import os

In [2]:
tab_file = pd.read_csv('./example.tab',sep='\t',header=None)
tab_file.columns = ['Family_HMM','HMM_length','Query_ID','Query_length','E_value',
                    'HMM_start','HMM_end','Query_start','Query_end','Coverage']
tab_file.head()

,Family_HMM,HMM_length,Query_ID,Query_length,E_value,HMM_start,HMM_end,Query_start,Query_end,Coverage
0,CBM32.hmm,124,KIY24080.1,150,2.900000e-11,18,115,29,135,0.782258
1,GT4.hmm,160,KIY24202.1,349,1.500000e-19,5,154,160,305,0.931250
2,GT8.hmm,257,KIY24250.1,315,8.200000e-55,30,255,3,246,0.875486
3,GT32.hmm,90,KIY24293.1,314,1.300000e-13,2,79,89,161,0.855556
4,GT4.hmm,160,KIY24299.1,348,9.000000e-22,7,150,170,309,0.893750


In [3]:
with open('example.faa','r') as f:
    seq_file = f.read()
seq_file

">KIY23963.1 hypothetical protein SB57_11565, partial [Lactobacillus delbrueckii subsp. bulgaricus]\nAEKSTKKAAKDTKKAAKTTKKSTKKAAKDTKKAAKTTKKSTKKAAKDTKKAAKTTKKSTKKAAKNTKKAAKTTKKSTKKA\nAKDTKKAAKTTKKSTKKA\n>KIY23964.1 hypothetical protein SB57_11555, partial [Lactobacillus delbrueckii subsp. bulgaricus]\nMTKNKSKVVFTLKQYVQAAKNTKTAKKTTKKAKAKQISVTYTLKDTTKAKNKQTLAKKTFKVKKGTSVFTVLKKAWKVNY\nTKSSKYGVFITKIKGLGDEKKKLYWTYTVDGKMAKVAADKQKLTKNKSKVVFTLKQ\n>KIY23965.1 transposase, partial [Lactobacillus delbrueckii subsp. bulgaricus]\nTEKLLSYGLTLKAVAWLTGLDKMTVKSIHKSLLLSKFTTDGKTLIKPERQAKYLAIDEFKLHDGHKYATAIIDWETGYIL\nYLAHGKKKAVVFEFMDHIGDEWMSKVEAVACDMNSDFEEAFKGRYPKLDIVYDRFHIIKNFNDKVVSEIRKDEQRRLAEE\nGSPEAAKLLKNSRYILMSKRSTLVQKDEDAR\n>KIY23966.1 membrane protein [Lactobacillus delbrueckii subsp. bulgaricus]\nMKSESKVSSKLELRELVLLAMVIAIKVILGQFKVGNATLQVGLGFIGSVMLGYLFGPWWGFAGGALSDLVSSXIFGNLGG\nFFIGFTLTAALXXMIYGFFLYKKPIQIWRVIASVICVTVICYIGLNTLWVSMLGGTNFMVALSSRILKEMITPWIHMVVV\nWFILEGLSRVKLSRKF\n>KIY23967.1 carbamoyl 

In [4]:
CAZYme_family = {'CAZyme':{'Enzyme_Classes':{'GlycosylTransferases':'GT',
              'GlycosideHydrolases':'GH',
              'CarbohydrateEsterases':'CE',
              'PolysaccharideLyases':'PL',
              'AuxiliaryActivities':'AA'},
              'Associated_Modules':{'Carbohydrate-BindingModules':'CBM'}}}

In [5]:
def find_value_recursive(cazy_dict,cazyme):
    global flag
    global levels
    flag = 0
    levels = cazyme
    for key,value in cazy_dict.items():
        if isinstance(value,dict):
            global n
            n += 1
            find_value_recursive(value, cazyme)
            if flag == 1:
                n -= 1
                levels =  'L'+str(n)+'_'+key+';'+levels
                return levels
            else:
                n -= 1
        elif cazyme == value:
            n += 1
            flag = 1
            levels =  'L'+str(n)+'_'+key+';'+'L'+str(n+1)+'_'+levels
            global n_level
            n_level = n+1
            return


In [6]:
if os.path.exists('example.fasta'):
    os.remove('example.fasta')
    
if os.path.exists('example_tax.txt'):
    os.remove('example_tax.txt')
        
for i in range(0,tab_file.shape[0]):
    global n,n_level
    n = 0
    n_level = 0
    family = tab_file.loc[i,'Family_HMM'].replace('.hmm','')
    query_id = tab_file.loc[i,'Query_ID']
    query_start = tab_file.loc[i,'Query_start']
    query_end = tab_file.loc[i,'Query_end']
    query_length = tab_file.loc[i,'Query_length']
    print('%d: %s'% (i+1,family))
    seq_pattern = re.compile(r'>'+query_id+'.*?\n(.*?)(>|\n$)',re.S)
    try:
        query_sequence = re.search(seq_pattern,seq_file).group(1).replace('\n','') #does |n count as char?
    except:
        print('Did not find %s in sequence!' % family)
        continue
    if(len(query_sequence)!=query_length):
        print("The length of %s did not match! Please check the input seqs!" % family)
        continue
    print('Query Sequence: %s'%query_sequence)
    
    target_sequence = query_sequence[query_start-1:query_end]
    print('Target Seuqunce: %s\n'%target_sequence)
    
    with open('example.fasta','a') as f:
        f.write('>cazy_%04d_%s\n%s\n'%(i,query_id,target_sequence))
    

    
    name_pattern = re.compile(r'(GH|GT|CBM|PL|AA|CE)(\d+)(_\d+)?')
    try:
        cazy_cat = re.search(name_pattern,family).group(1)
        cazy_num = re.search(name_pattern,family).group(2)
    except:
        print('Find %s!'%family)
        print("Interesting family! What is that?")
        continue
        
        
    cazy_prefix = find_value_recursive(CAZYme_family,cazy_cat)
    cazy_tax = cazy_prefix+cazy_num
    #print(cazy_tax)
    print(n_level)
    
    if cazy_cat == 'GH' and int(cazy_num) in [5,13,30,43]:
        try:
            cazy_sub_num = re.search(name_pattern,family).group(3)
        except:
            print('%s%s had no sub family.'%(cazy_cat,cazy_num))
        cazy_tax = cazy_tax+';L'+str(n_level+1)+'_'+family
        
    print(cazy_tax)
    print('\n')
    
    with open('example_tax.txt','a') as f:
        f.write('cazy_%04d_%s\t%s\n'%(i,query_id,cazy_tax))
    

    


1: CBM32
Query Sequence: MQVKTKRDPRDLSSPMSKSLPTRKRKRPPAETTLDRRMATDWQSTGKPSEDQPLTLTFSFPDQTELNKLIYVPRASSRDGRPLRMRVEISTDGENFTSYGTDYHFKSDCKNKVIGLRGVSAKAVRLSILETTGDYATASGIDFYRPLNKP
Target Seuqunce: PAETTLDRRMATDWQSTGKPSEDQPLTLTFSFPDQTELNKLIYVPRASSRDGRPLRMRVEISTDGENFTSYGTDYHFKSDCKNKVIGLRGVSAKAVRLSILETTGDY

4
L1_CAZyme;L2_Associated_Modules;L3_Carbohydrate-BindingModules;L4_CBM32


2: GT4
Query Sequence: MIRINMFSQADSVAGQGVGAAYLELIRLLRTHLVDDFYVTINKYGRSDLTHYHTINPTYFANSFSPARGRKIGYVHFLPDTLERSIKLPGVAKKVFYQYVIDFYKRMDQIVVVNPIFIDKLTEYGLDRNKIRYIPNFVAKSEFYEQTQAKKNATRQELGIPQDKFVVFGDGQVQARKGVDDFAKLAEANPDYQFIWAGGFSFGKITDGYDHFKQLVENPPKNLTFTGIIDREKLVDYLNMADLFLLPSFDELFPMSVLEAFSCGTPVLLRDLDLYRAIIDGYYLAGKDSAEMNEQLRFAATHPEVLARQQELSRKASQQYSEDHLAEIWRDFYQEQYQLGRELGQISYE
Target Seuqunce: IPQDKFVVFGDGQVQARKGVDDFAKLAEANPDYQFIWAGGFSFGKITDGYDHFKQLVENPPKNLTFTGIIDREKLVDYLNMADLFLLPSFDELFPMSVLEAFSCGTPVLLRDLDLYRAIIDGYYLAGKDSAEMNEQLRFAATHPEV

4
L1_CAZyme;L2_Enzyme_Classes;L3_GlycosylTransferases;L4_GT4


3: GT8
Query Sequence